In [2]:
import csv
import copy
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
market_csv_file = "data/S&P500.csv"
stock_csv_file = "data/stock_clean.csv"
df_market = pd.read_csv(market_csv_file)
df_stock = pd.read_csv(stock_csv_file)

In [4]:
df_stock = df_stock.rename(columns = {df_stock.columns[0]:"date"})
result = df_stock["date"].isin(["19720731"])
t = df_stock["date"][result].index.values[0]
print(t)

119


In [5]:
#Drop columns that have less than pre 120 + post 12 = 132 valid return
tmp_df_stock_all = df_stock[t-119:t+13].dropna(thresh=df_stock[t-119:t+13].shape[0],how='all',axis=1)
#print(tmp_df_stock_all)
#Get the prev
tmp_df_stock = tmp_df_stock_all[0:120]
tmp_df_market = df_market[t-119:t+1]
N = len(tmp_df_stock.columns[1:])

In [6]:
#sample covariance matrix estimator
sample_cov = tmp_df_stock[tmp_df_stock.columns[1:]].astype(np.float64).cov()

In [46]:
#market model estimator
y = tmp_df_stock[tmp_df_stock.columns[1:]].astype(np.float64).values
y_bar = y.mean(axis=0)
x = tmp_df_market[tmp_df_market.columns[1:]].astype(np.float64).values
x_bar = x.mean()
beta_1 = np.sum((y-y_bar)*(x),axis=0)/np.sum((x-x_bar)*x,axis=0)
beta_0 = y_bar - beta_1*x_bar
residual = y - (beta_1*x+beta_0)
delta = np.sum(residual**2,axis=0)/(N-2)
D = np.diag(delta)
s_2 = x.var()
print(s_2)
F = s_2*beta_1[:,np.newaxis]@beta_1[:,np.newaxis].T + D
S = tmp_df_stock[tmp_df_stock.columns[1:]].astype(np.float64).cov()
#print(F)

0.001261002278338264


In [66]:
pi = 0
rou = 0
gamma = 0
T = 120

# for i in range(N):
#     for j in range(N):
#         gamma += (F[i][j]-S[i][j])**2
        
#         tmp = 0
#         for k in range(T):
#             tmp += ((y[k][i]-y_bar[i])*(y[k][j]-y_bar[j])-S[i][j])**2
#         pi += tmp/T
        
#         if i == j:
#             rou += tmp/T
#         else:
#             stock_i = y[:,[i]]
#             stock_j = y[:,[j]]
#             s_i0 = cov()
            
#             for k in range(T):
yx = np.concatenate((y,x),axis=1)
yx_cov = np.cov(yx,rowvar=False)
gamma = ((F-S)**2).sum().sum()
s_x0 = np.expand_dims(yx_cov[-1,:-1],axis=1)
PI = ((((y-y_bar).T@(y-y_bar)-S)**2)/T).values
pi = PI.sum().sum()
print(pi)
print(gamma)
r = np.zeros((N,N))
s_00 = yx_cov[-1,-1]
for i in range(T):
    y_e = np.expand_dims(y[i,:]-y_bar,axis=1)
    r+=(x[0]-x_bar)*(s_00*s_x0@y_e.T-(s_00*y_e@s_x0.T-s_x0@s_x0.T)*(y_e@y_e.T))/(s_00**2) - F*S
print(type(r))
np.fill_diagonal(r.values,np.diag(PI))
rou =r.values.sum().sum()
print(rou)
k = (pi-rou)/gamma
#print(k)
E = k/T * F +(T-k)/T* S
print(E)
print(yx_cov[-1,-1])
print(yx.shape)
print(yx_cov.shape)

620.5031425396816
0.8436649542474596
<class 'pandas.core.frame.DataFrame'>
2834.3425247079363
          ACF        AJ       ADX        AB        AN       AYP       ACD  \
ACF  0.079479  0.015678  0.003908  0.011528  0.005842  0.000611  0.000596   
AJ   0.015678  0.285846  0.011009  0.069170  0.016854 -0.012981  0.017369   
ADX  0.003908  0.011009  0.026739  0.008408  0.003170  0.004162  0.002373   
AB   0.011528  0.069170  0.008408  0.127690  0.019070 -0.019134  0.013528   
AN   0.005842  0.016854  0.003170  0.019070  0.078957 -0.001595  0.032452   
..        ...       ...       ...       ...       ...       ...       ...   
CLI  0.005496  0.033335 -0.002754  0.028230  0.009050 -0.002338  0.018653   
WCP  0.015480  0.021652  0.000316  0.015175  0.001492 -0.001324  0.000068   
MPO -0.021920  0.132078  0.017294  0.078632  0.004708 -0.021191  0.024239   
SG   0.013840  0.096114  0.013950  0.074362  0.020649  0.002139 -0.007568   
BFD -0.003123 -0.019308 -0.000528  0.006514 -0.001369  0.00

In [62]:
#backtest
monthly_returns = []
#range(t,384,12)
for i in tqdm(range(t,t+1,12)):

    #Drop columns that have less than pre 120 + post 12 = 132 valid return
    tmp_df_stock_all = df_stock[i-119:i+13].dropna(thresh=df_stock[i-119:i+13].shape[0],how='all',axis=1)

    #In sample
    tmp_df_stock = tmp_df_stock_all[0:120]
    N = len(tmp_df_stock.columns[1:])
    tmp_df_market = df_market[t-119:t+1]
    
    #sample covariance matrix estimator
    sample_cov = tmp_df_stock[tmp_df_stock.columns[1:]].astype(np.float64).cov()
    
    #market model estimator
    y = tmp_df_stock[tmp_df_stock.columns[1:]].astype(np.float64).values
    y_bar = y.mean(axis=0)
    x = tmp_df_market[tmp_df_market.columns[1:]].astype(np.float64).values
    x_bar = x.mean()
    beta_1 = np.sum((y-y_bar)*(x-x_bar),axis=0)/np.sum((x-x_bar)*(x-x_bar),axis=0)
    beta_0 = y_bar - beta_1*x_bar
    residual = y - (beta_1*x+beta_0)
    delta = np.sum(residual**2,axis=0)/(N-2)
    D = np.diag(delta)
    s_2 = x.var()
    F = s_2*beta_1[:,np.newaxis]@beta_1[:,np.newaxis].T + D
    
    cov_matrix = F
    #cov_matrix_inv = pd.DataFrame(np.linalg.pinv(cov_matrix.values), cov_matrix.columns, cov_matrix.index)
    cov_matrix_inv = np.linalg.pinv(cov_matrix)
    #print(cov_matrix_inv)
    
    w = cov_matrix_inv@np.ones((N,1))/(np.ones((1,N))@cov_matrix_inv@np.ones((N,1)))
    mu = tmp_df_stock[tmp_df_stock.columns[1:]].mean()
    #print(mu)
    
    for j in range(12):
        tmp_df_post = tmp_df_stock_all[120+j:121+j]
        tmp_df_post = tmp_df_post[tmp_df_post.columns[1:]]
        #print(tmp_df_post)
        monthly_return = (tmp_df_post.astype(np.float64)+1).prod(axis=0)-1
        monthly_returns.append(monthly_return@w)
    

100%|██████████| 23/23 [00:06<00:00,  3.57it/s]


In [63]:
monthly_returns = np.array(monthly_returns)
monthly_returns.std()*np.sqrt(12)

0.1432967768232372

In [ ]:
a = s

(120, 811)
(811, 811)


array([[0.01595323, 0.00163098],
       [0.00163098, 0.00228553]])